In [3]:
import tensorflow as tf
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.xception import Xception
import segmentation_models as sm
import tensorflow_advanced_segmentation_models as tasm
import numpy as np
import visualkeras

## Failed ResNet Test

In [2]:
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(512, 512, 3))

In [10]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [28]:
layer_names = ["conv1_pad", "conv1_conv", "conv1_bn", "conv1_relu", "pool1_pad" "pool1_pool"]

resnet_blocks = {"conv7": ["conv1_pad", "conv1_conv", "conv1_bn", "conv1_relu", "pool1_pad", "pool1_pool"],
                 "block_3" : ["conv2_block3_1_conv", "conv2_block3_1_bn", "conv2_block3_1_relu", "conv2_block3_2_conv", "conv2_block3_2_bn", "conv2_block3_2_relu"],
                 "block_4" : ["conv3_block4_1_conv", "conv3_block4_1_bn", "conv3_block4_1_relu", "conv3_block4_2_conv", "conv3_block4_2_bn", "conv3_block4_2_relu"],
                 "block_6" : ["conv4_block6_1_conv", "conv4_block6_1_bn", "conv4_block6_1_relu", "conv4_block6_2_conv", "conv4_block6_2_bn", "conv4_block6_2_relu"],
                 "block_3_2" : ["conv5_block3_1_conv", "conv5_block3_1_bn", "conv5_block3_1_relu", "conv5_block3_2_conv", "conv5_block3_2_bn", "conv5_block3_2_relu"]}

In [47]:
layers = [resnet.get_layer(layer_name) for layer_name in resnet_blocks["block_3"]]

In [45]:
x = tf.keras.layers.Input((512, 512, 3))
copy = x
for layer in layers:
    x = layer(x)
#output = tf.keras.layers.Add()([copy, x])

In [46]:
x.shape

TensorShape([None, 128, 128, 64])

In [48]:
for layer in layers:
    x = layer(x)

ValueError: Exception encountered when calling layer "conv2_block3_1_conv" (type Conv2D).

Depth of input (64) is not a multiple of input depth of filter (256) for '{{node conv2_block3_1_conv/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2_block3_1_conv/Conv2D/ReadVariableOp)' with input shapes: [?,128,128,64], [1,1,256,64].

Call arguments received by layer "conv2_block3_1_conv" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 128, 128, 64), dtype=float32)

## Xception

In [57]:
x = Xception(weights="imagenet", input_shape=(512,512, 3), include_top=False)

In [58]:
x.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 255, 255, 32  864         ['input_18[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 255, 255, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

## VGG U-Net

In [4]:
vgg19 = VGG19(weights="imagenet", include_top=False, input_shape=(512,512,3))

In [5]:
visualkeras.layered_view(vgg19).show()

In [6]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 512, 512, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 512, 512, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 256, 256, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 256, 256, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 256, 256, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 128, 128, 128)     0     

In [7]:
#Getting all the blocks from the VGG network
vgg_blocks = {
    f"block{n}" : [layer for layer in vgg19.layers if f"block{n}_conv" in layer.name] for n in range(1, 6)
}

In [8]:
vgg_blocks

{'block1': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb21ef76990>,
 'block2': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb21f429790>,
 'block3': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb21f43a790>,
 'block4': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb21f4b9950>,
 'block5': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb21f4d8910>,
  <keras.layers.convolutional.conv2d.Conv2D at 0x7fb21f4d8950>]}

### VGG 1 - Encoder

In [9]:
def vgg_encoder_block(x, layers):
    """
    This function passes an input through a set of conv layers from VGG19, returning the downsampled and convolved activation
    """
    for layer in layers:
        x = layer(x)
    
    addition = x
    x = tf.keras.layers.MaxPooling2D((2,2), strides = 2)(x)
    return (x, addition)

def last_vgg_block(x, layers):

    for layer in layers:
        x = layer(x)
    
    return x

In [10]:
input_x = vgg19.input

In [11]:
input_x

<KerasTensor: shape=(None, 512, 512, 3) dtype=float32 (created by layer 'input_1')>

In [12]:
block_1 = vgg_encoder_block(input_x, vgg_blocks["block1"])

In [13]:
block_1

(<KerasTensor: shape=(None, 256, 256, 64) dtype=float32 (created by layer 'max_pooling2d')>,
 <KerasTensor: shape=(None, 512, 512, 64) dtype=float32 (created by layer 'block1_conv2')>)

In [14]:
def vgg_encoder_full(input, layer_dict):

    """
    This function creates the full encoder given a dictionary of layers from the VGG network, it returns the final activation 
    and a list of intermediate activations
    """

    activations = []
    x = input
    for layer_name in list(layer_dict.keys())[:-1]:
        x, a = vgg_encoder_block(x, layer_dict[layer_name])
        activations.append(a)
    
    x = last_vgg_block(x, layer_dict[list(layer_dict.keys())[-1]])
    
    return x, activations

In [15]:
x, a = vgg_encoder_full(input_x, vgg_blocks)

In [16]:
x

<KerasTensor: shape=(None, 32, 32, 512) dtype=float32 (created by layer 'block5_conv4')>

### VGG 2 - Decoder

In [17]:
def decoder_block(a, x, f):

    x = tf.keras.layers.Conv2DTranspose(filters=f, kernel_size=2, strides=2, padding="same", activation="relu")(x)
    x = tf.concat([a, x], axis=-1)
    x = tf.keras.layers.Conv2D(f, 3, padding="same", activation="relu")(x)
    x = tf.keras.layers.Conv2D(f, 3, padding="same", activation="relu")(x) 

    return x

In [18]:
def decoder_full(activations, x, filters, num_classes):

    for a,f in zip(activations[::-1],filters):
        x = decoder_block(a, x, f)
    
    output = tf.keras.layers.Conv2D(num_classes, 1, padding="same", activation="softmax")(x)

    return output

In [19]:
filters = [512, 256, 128, 64]
o = decoder_full(a, x, filters, 3)

In [20]:
o

<KerasTensor: shape=(None, 512, 512, 3) dtype=float32 (created by layer 'conv2d_8')>

### VGG 3 - Full U-Net


In [21]:
def vgg_unet(num_classes, input_size, input_dim):

    #Downloading the VGG network
    vgg19 = VGG19(weights="imagenet", include_top=False, input_shape=(input_size, input_size,input_dim))
    vgg19.trainable = False
    #Getting all the blocks from the VGG network
    vgg_blocks = {
        f"block{n}" : [layer for layer in vgg19.layers if f"block{n}_conv" in layer.name] for n in range(1, 6)
    }
    
    #Filters for the Decoder
    filters = [512, 256, 128, 64]

    vgg_input = vgg19.input

    #Defining the encoder

    #First Preprocess the input
    x = preprocess_input(x=vgg_input)
    
    x, a = vgg_encoder_full(x, vgg_blocks)

    output = decoder_full(a, x, filters, num_classes)

    vgg_unet_model = tf.keras.Model(vgg_input, output)

    return vgg_unet_model
    

    

    

In [22]:
vnet = vgg_unet(3, 512, 3)

In [27]:
visualkeras.layered_view(vnet, legend=True, scale_xy=0.5).show()

In [23]:
vnet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 512, 512, 3)  0          ['input_2[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 tf.nn.bias_add (TFOpLambda)    (None, 512, 512, 3)  0           ['tf.__operators__.getitem[0][0]'
                                                                 ]                            

## ResNet-U-Net

In [28]:
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(512,512,3))

In [29]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [30]:
resnet_blocks = {
    f"block_{n}" : [layer for layer in resnet.layers if layer.name[:5] == f"conv{n}"] for n in range(1, 5)
}

In [31]:
resnet_outputs = {
    f"output_{n}" : [layer.name for layer in resnet.layers if layer.name[:5] == f"conv{n}" and "out" in layer.name][-1] for n in range(2, 5)
}

In [32]:
size = 0
for k in resnet_blocks.keys():
    size += len(resnet_blocks[k])
print(f"Number of layers {size}")

Number of layers 140


In [33]:
su = [1 for layer in resnet.layers]

In [34]:
sum(su)

175

### Encoder of the ResNet-U-Net

In [35]:
resnet_blocks

{'block_1': [<keras.layers.reshaping.zero_padding2d.ZeroPadding2D at 0x7fb216c57c10>,
 'block_2': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb2224c6750>,
 'block_3': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb2224bb8d0>,
 'block_4': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb20e984610>,
  <keras.layers.core.activation.Activation at 0x7fb20e996490>]}

In [36]:
def resblock_stem(input, layers):

    x = input
    for layer in layers:

        x = layer(x)
    
    a = x
    x = tf.keras.layers.MaxPool2D(2, 2)(x)

    return x, a


def resblock_enc(inp, layers):

    enc = tf.keras.Model(inputs=resnet.get_layer(layers[0].name).input, outputs=resnet.get_layer(layers[-1].name).output)
    x = enc(inp)
    return x

def resnet_encoder(inp, layer_dict):

    keys = list(layer_dict.keys())

    x, a = resblock_stem(inp, layer_dict[keys[0]])

    activations = [a]
    for block in keys[1:]:
        x = resblock_enc(x, layer_dict[block]) 
        activations.append(x)
    
    return x, [None] + activations[:-1]



In [37]:
inp = resnet.input

In [38]:
x, a = resblock_stem(inp, resnet_blocks["block_1"])

In [39]:
x

<KerasTensor: shape=(None, 128, 128, 64) dtype=float32 (created by layer 'max_pooling2d_9')>

In [40]:
x2 = resblock_enc(x, resnet_blocks["block_2"])

In [41]:
x2

<KerasTensor: shape=(None, 128, 128, 256) dtype=float32 (created by layer 'model_1')>

In [42]:
x, a = resnet_encoder(inp, resnet_blocks)

### Decoder for The ResNet-Unet

The last decoder block of this U-Net will not use an activation

In [43]:
def decoder_block(a, x, f):

    x = tf.keras.layers.Conv2DTranspose(filters=f, kernel_size=2, strides=2, padding="same", activation="relu")(x)
    if a is not  None:
        x = tf.concat([a, x], axis=-1)
    x = tf.keras.layers.Conv2D(f, 3, padding="same", activation="relu")(x)
    x = tf.keras.layers.Conv2D(f, 3, padding="same", activation="relu")(x) 

    return x

def decoder_full(activations, x, filters, num_classes):

    for a,f in zip(activations[::-1],filters):
        x = decoder_block(a, x, f)
    
    output = tf.keras.layers.Conv2D(num_classes, 1, padding="same", activation="softmax")(x)

    return output

In [44]:
filters = [512, 256, 128, 64]

In [45]:
o = decoder_full(a, x, filters, 5)

### Full ResNet-Unet

In [46]:
def resblock_stem(input, layers):

    x = input
    for layer in layers:

        x = layer(x)
    
    a = x
    x = tf.keras.layers.MaxPool2D((2,2), 2)(x)

    return x, a


def resblock_enc(inp, layers, res):

    enc = tf.keras.Model(inputs=res.get_layer(layers[0].name).input, outputs=res.get_layer(layers[-1].name).output)
    x = enc(inp)
    return x

def resnet_encoder(inp, layer_dict, res):

    keys = list(layer_dict.keys())

    x, a = resblock_stem(inp, layer_dict[keys[0]])

    activations = [a]
    for block in keys[1:]:
        x = resblock_enc(x, layer_dict[block], res) 
        activations.append(x)
    
    return x, [None] + activations[:-1]



def resnet_unet(num_classes, input_size, input_dim):

    #Downloading the ResNet
    resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(input_size,input_size,input_dim))

    layer_dict = resnet_blocks = {
    f"block_{n}" : [layer for layer in resnet.layers if layer.name[:5] == f"conv{n}"] for n in range(1, 5)
    }

    #Freezing the layers of the ResNet
    resnet.trainable = False

    #Building the model
    inp = resnet.input
    x = preprocess_input(x=inp)
    x, a = resnet_encoder(inp, layer_dict, resnet)

    filters = [512, 256, 128, 64]

    output = decoder_full(a, x, filters, num_classes)

    model = tf.keras.Model(inp, output)

    return model

In [47]:
m = resnet_unet(5, 512, 3)

In [49]:
visualkeras.layered_view(m, scale_xy=1, legend=True).show()

In [48]:
m.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_8[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[1][0]']              
                                )                                                                 
                                                                                            

## Efficient-Net

In [50]:
from tensorflow.keras.applications.efficientnet import EfficientNetB4

In [51]:
b4 = EfficientNetB4(weights="imagenet", include_top=False, input_shape=(512,512,3))

In [52]:
b4.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_12[0][0]']               
                                                                                                  
 normalization (Normalization)  (None, 512, 512, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 512, 512, 3)  0           ['normalization[0][0

In [53]:
effnet_blocks = {}
stem_start = [b4.get_layer("rescaling"), b4.get_layer("normalization"), b4.get_layer("rescaling_1")]
stem_start.extend([layer for layer in b4.layers if layer.name[:4] == "stem"])
effnet_blocks["stem"] = stem_start
effnet_blocks = {**effnet_blocks, **{ 
    f"block_{n}" : [layer for layer in b4.layers if layer.name[:6] == f"block{n}"] for n in range(1, 8)
}}

In [54]:
effnet_blocks

{'stem': [<keras.layers.preprocessing.image_preprocessing.Rescaling at 0x7fb20127c150>,
 'block_1': [<keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D at 0x7fb1fdf730d0>,
 'block_2': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb1fe0d35d0>,
 'block_3': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb1f06a42d0>,
 'block_4': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb1f13b2ed0>,
 'block_5': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb1f284ef10>,
 'block_6': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb1f2c7d050>,
 'block_7': [<keras.layers.convolutional.conv2d.Conv2D at 0x7fb1f5916d90>,
  <keras.layers.merging.add.Add at 0x7fb1f5dc9810>]}

### Efficient-Net Encoder

In [55]:
def effnet_stem(input, layers):

    x = input

    for layer in layers:
        x = layer(x)
    
    return x
    
def effblock_enc(inp, layers, model):

    enc = tf.keras.Model(inputs=model.get_layer(layers[0].name).input, outputs=model.get_layer(layers[-1].name).output)
    x = enc(inp)
    return x

def effnet_encoder(inp, layer_dict, model):

    keys = list(layer_dict.keys())

    x = effnet_stem(inp, layer_dict[keys[0]])

    activations = [x]
    for block in keys[1:]:

        x = effblock_enc(x, layer_dict[block], model) 
        if block == "block_1" or block ==  "block_4" or block == "block_6":
            continue
        activations.append(x)
    
    return x, [None] + activations[:-1]


#### Individual Encoder Tests

In [56]:
s = effnet_blocks["stem"]

In [57]:
inp = b4.input
x = effnet_stem(inp, effnet_blocks["stem"])
x

<KerasTensor: shape=(None, 256, 256, 48) dtype=float32 (created by layer 'stem_activation')>

In [58]:
x = effblock_enc(x, effnet_blocks["block_1"], b4)
x

<KerasTensor: shape=(None, 256, 256, 24) dtype=float32 (created by layer 'model_9')>

#### Full Encoder Test

In [59]:
x, a = effnet_encoder(inp, effnet_blocks, b4)

In [60]:
x

<KerasTensor: shape=(None, 16, 16, 448) dtype=float32 (created by layer 'model_16')>

In [61]:
a[::-1]

[<KerasTensor: shape=(None, 32, 32, 160) dtype=float32 (created by layer 'model_14')>,
 <KerasTensor: shape=(None, 64, 64, 56) dtype=float32 (created by layer 'model_12')>,
 <KerasTensor: shape=(None, 128, 128, 32) dtype=float32 (created by layer 'model_11')>,
 <KerasTensor: shape=(None, 256, 256, 48) dtype=float32 (created by layer 'stem_activation')>,
 None]

### Efficient-Net U-Net Full

In [62]:
def effnet_unet(num_classes, input_size, input_dim):

    b4 = EfficientNetB4(weights="imagenet", include_top=False, input_shape=(input_size,input_size,input_dim))

    effnet_blocks_dict = {}
    stem_start = [b4.layers[0], b4.layers[1], b4.layers[2]]
    stem_start.extend([layer for layer in b4.layers if layer.name[:4] == "stem"])
    effnet_blocks_dict["stem"] = stem_start
    effnet_blocks_dict = {**effnet_blocks, **{ 
        f"block_{n}" : [layer for layer in b4.layers if layer.name[:6] == f"block{n}"] for n in range(1, 8)
    }}

    #Freeszing the weights of the model
    b4.trainable = False

    #Encoder of the network
    inp = b4.input
    x = preprocess_input(inp)
    #a not being reversed here because it's reversed in the decoder function
    x, a = effnet_encoder(x, effnet_blocks_dict, b4)

    #Decoder of the network
    filters = [160, 56, 32, 48, 64]
    
    output = decoder_full(a, x, filters, num_classes)

    model  = tf.keras.Model(inp, output)

    return model

    

In [63]:
m = effnet_unet(5, 512, 3)

In [64]:
visualkeras.layered_view(m, scale_xy=1, legend=True).show()

: 

In [145]:
m.summary()

Model: "model_56"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_53 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 tf.__operators__.getitem_1 (Sl  (None, 512, 512, 3)  0          ['input_53[0][0]']               
 icingOpLambda)                                                                                   
                                                                                                  
 tf.nn.bias_add_1 (TFOpLambda)  (None, 512, 512, 3)  0           ['tf.__operators__.getitem_1[0][0
                                                                 ]']                       

## Base U-Net

In [178]:
def encoder_block(inp,f):

    x = inp

    x = tf.keras.layers.Conv2D(f, 3, 1, padding="same")(x)
    x = tf.keras.layers.Conv2D(f, 3, 1, padding="same")(x)
    a = x
    x = tf.keras.layers.MaxPool2D(2, 2)(x)

    return x,a

def last_encoder(inp, f):

    x = inp

    x = tf.keras.layers.Conv2D(f, 3, 1, padding="same")(x)
    x = tf.keras.layers.Conv2D(f, 3, 1, padding="same")(x)

    return x

In [179]:
def encoder_unet(inp, filters):

    activations = []

    x = inp
    for f in filters[:-1]:
        x, a = encoder_block(x, f)
        activations.append(a)
    
    x = last_encoder(x, filters[-1])
    return x, activations

In [180]:
encoder_unet(tf.keras.layers.Input((512, 512, 3)), [64, 128, 256, 512, 1024])

(<KerasTensor: shape=(None, 32, 32, 1024) dtype=float32 (created by layer 'conv2d_111')>,
 [<KerasTensor: shape=(None, 512, 512, 64) dtype=float32 (created by layer 'conv2d_103')>,
  <KerasTensor: shape=(None, 256, 256, 128) dtype=float32 (created by layer 'conv2d_105')>,
  <KerasTensor: shape=(None, 128, 128, 256) dtype=float32 (created by layer 'conv2d_107')>,
  <KerasTensor: shape=(None, 64, 64, 512) dtype=float32 (created by layer 'conv2d_109')>])

In [181]:
def unet(num_classes, input_size, input_dim):

    inp = tf.keras.layers.Input((input_size, input_size, input_dim))

    filters = [64, 128, 256, 512, 1024]

    x, a = encoder_unet(inp, filters=filters)

    o = decoder_full(a, x, filters, num_classes)

    model = tf.keras.Model(inp, o)

    return model

In [182]:
un = unet(3, 512, 3)

In [183]:
un.summary()

Model: "model_49"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_60 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_112 (Conv2D)            (None, 512, 512, 64  1792        ['input_60[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_113 (Conv2D)            (None, 512, 512, 64  36928       ['conv2d_112[0][0]']             
                                )                                                          

## Xception-U-Net - Scrapped

In [184]:
xception = Xception(weights="imagenet", include_top=False, input_shape=(512,512,3))

In [185]:
xception.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_61 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 255, 255, 32  864         ['input_61[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 255, 255, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

## Efficient-Net V2

In [186]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2M

In [187]:
efM = EfficientNetV2M(weights="imagenet", include_top=False, input_shape=(512, 512, 3))

214201816/214201816 [==============================] - 16s 0us/step


In [188]:
efM.summary()

Model: "efficientnetv2-m"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_62 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_6 (Rescaling)        (None, 512, 512, 3)  0           ['input_62[0][0]']               
                                                                                                  
 stem_conv (Conv2D)             (None, 256, 256, 24  648         ['rescaling_6[0][0]']            
                                )                                                                 
                                                                                   